# **2022년 빅데이터 동아리 특강 - 토픽 모델링 (2022년 5월 23일)**

대학원 기술경영학과 (Management of Technology) - 송지훈 교수

🧑👩 학생 여러분, 빅데이터 동아리에 오신걸 환영합니다 !\
짧은 시간이지만, 여러분들이 다양한 특강을 기반으로 **스스로 학습** 및 **경진대회**에 **참가** 할 수 있는 **역량**을 갖출 수 있도록 지원하는게 주 목적 입니다.

**나중을 위한 팁**\
✅ You can only learn data science by doing data science. (실제로 코드를 구현해 봐야 합니다 ~) \
✅ Practice, practice, practice. (연습하고 또 연습하세요, 이번 짧은 강의에서는 모든 세세한 내용을 전부 다룰수 없습니다 ~)\
✅ Free resources everywhere. (인터넷상에는 무료로 데이터 분석 또는 프로그래밍 관련 공부를 할 수 있는 많은 자료들이 존재 합니다. 적극적으로 찾아서 이용하세요 ~)

# **한글처리를 위한 기본 세팅**

In [ ]:
!apt -qq -y install fonts-nanum

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
sys_font=fm.findSystemFonts()
nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}")
print(nanum_font)

In [ ]:
path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
fm._rebuild()

In [ ]:
# ======= 형태소 분석을 위해 한글 분석 모듈 konlpy를 설치한다. =============
!python -m pip install konlpy
import konlpy 
print('KoNLPy version...:', konlpy.__version__)

In [ ]:
# 맞춤법을 고치기 위한 라이브러리 설치
# 코드를 실행 후 재시작 필요
!pip install git+https://github.com/ssut/py-hanspell.git 

In [ ]:
# Mecab이 필요한 경우에만 사용
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# **데이터 크롤링 부분 (구글 플레이 스토어)**

In [ ]:
!pip install selenium # 셀레니움
!apt-get update 
!apt install chromium-chromedriver

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import random
import time

약간의 문제가 있는 코드, 로컬 개발 환경에서는 정상 작동.

In [ ]:
# ---- 셀레니움 각종 함수들 불러오기
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

# ---- 옵션 설정 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') # 
chrome_options.add_argument('--disable-dev-shm-usage')

# ----- 웹드라이버 로딩
driver = webdriver.Chrome('chromedriver', options=chrome_options) # Colab 환경이 아닌 경우에는, chromedriver 파일을 컴퓨터에서 실행시켜야 함.
driver.maximize_window() # 크롬창 크기 최대

# 드라이버가 해당 url 접속
url = 'https://play.google.com/store/apps/details?id=com.towneers.www&hl=ko&gl=US&showAllReviews=true' # 
driver.get(url)
print(driver.title)
time.sleep(2)
SCROLL_PAUSE_TIME = 2
SCROLL_TIMES = 5 # 
CLICK_PAUSE_TIME = 2

# 리뷰 페이지의 마지막까지 스크롤 다운하기 위해 페이지의 높이를 return
last_height = driver.execute_script("return document.body.scrollHeight")

# 스크롤 가장 아래까지 내리기 ('더보기' 누르면서)
#while True:
for k in range(30):
    print(k)
    
    for i in range(SCROLL_TIMES): 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME) # 스크롤 다운 사이에 2초의 시간(pause)을 두어 에러를 방지
        print("내려감")
    
    more_button = driver.find_elements(By.XPATH,"//span[@class='RveJvd snByac']")
    
    # '더 보기' 버튼이 있다면 눌러준다
    if more_button:
        more_button[0].click()
        print("clicked")
    time.sleep(1)
    # 더이상 내려가는 곳이 없으면 break
    new_height = driver.execute_script("return document.body.scrollHeight")
    print(new_height)

    if new_height == last_height:
        print(last_height)
        break

    last_height = new_height
    time.sleep(1)    

In [ ]:
# 총 리뷰 확인
reviews = driver.find_elements(By.XPATH,'//*[@jsname="fk8dgd"]//div[@class="d15Mdf bAhLNe"]')
print(f'총 {len(reviews)} 리뷰를 획득했다!')

In [ ]:
# 앱 이름
app_name = driver.find_element(By.CLASS_NAME, "AHFaub")
앱 = app_name.text
print(앱)
app_collector = [앱 for i in range(len(reviews))]

# 유저 이름
user_name = driver.find_elements(By.XPATH, '//div[@class="bAhLNe kx8XBd"]/span[@class="X43Kjb"]') 
print(f"총 스크래핑한 유저의 수는: {len(user_name)}") # (10번 loop)
name_collector = []
for name in user_name:
    name_collector.append(name.text)
    
# 리뷰 날짜
review_collector = []
for i in range(1,len(reviews)+1):
    review_date = driver.find_element(By.XPATH, f'//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[1]/div[1]/div/span[2]') 
    review_collector.append(review_date.text)

# 평점 
score = driver.find_elements(By.XPATH,'//div[@class="pf5lIe"]/div[@role="img"]')
score_collector = []
for e in score:
    score_collector.append(e.get_attribute('aria-label').replace('별표 5개 만점에', '').replace('개를 받았습니다.', '').strip()) # first one is weird
    
final_score = score_collector[1:]

# 좋아요 (리뷰가 도움이 되었는지에 대한 여부)
likes = driver.find_elements(By.XPATH,'//div[@class="jUL89d y92BAb"]')
likes_collector =[]
for l in likes:
    likes_collector.append(l.text)

# 리뷰 내용
review = driver.find_elements(By.XPATH,"//span[@jsname='bN97Pc']")
print(len(review))
review_content = []
for j in review:
    review_content.append(j.text)

In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(app_collector,name_collector,review_collector, final_score,likes_collector,review_content)),
               columns =['앱이름','유저이름','리뷰날짜','평점','리뷰의 좋아요 개수','리뷰 내용'])
df    

df.to_csv("당근리뷰.csv", index=False, encoding="utf-8-sig")  

# **토픽 모델링 부분**

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/gnu-mot/student_club/main/%EB%8B%B9%EA%B7%BC%EB%A6%AC%EB%B7%B0.csv")
df.head()

In [ ]:
print(f'총 {len(df)} 리뷰를 획득했다!')

In [ ]:
df.info()

In [ ]:
# 중복값 확인
duplicateRows = df[df['리뷰 내용'].duplicated()]
duplicateRows # 중복값은 없음

## **텍스트 전처리**

In [ ]:
import re
def clean_text(text):  
    text = text.replace(".", " ").strip()
    text = re.sub('[^가-힣|a-zA-Z]+', ' ', text) # 한국어, 영어를 제외하고 나머지를 필터링
    text = re.sub(' +', ' ', text)
    return text

In [ ]:
df['전처리1'] = df['리뷰 내용'].apply(clean_text)

In [ ]:
df.head()

In [ ]:
# 맞춤법 교정
from hanspell import spell_checker
spelled_sent = spell_checker.check(df['전처리1'][1])
print(spelled_sent.checked) # 문법 교정 (주의: 너무 긴 문장의 경우, 정확도가 떨어지거나 제대로 작동하지 않음.)

In [ ]:
def spell_check(text):
    try:
        spelled_sent = spell_checker.check(text)
        text = spelled_sent.checked
    except:
        text = text
    return text

In [ ]:
df['spell_check'] = df['전처리1'].apply(spell_check) # 15분 정도 소요 !! (시간을 절약하기 위해 아래 내용이 포함된 파일을 준비)

In [ ]:
df.head()

In [ ]:
df.to_csv("당근리뷰_맞춤법교정.csv", index=False, encoding="utf-8-sig")  

## **문서에서 명사만 추출**

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/gnu-mot/student_club/main/%EB%8B%B9%EA%B7%BC%EB%A6%AC%EB%B7%B0_%EB%A7%9E%EC%B6%A4%EB%B2%95%EA%B5%90%EC%A0%95.csv")
data['spell_check'] = data['spell_check'].str.strip()
data.head()

In [ ]:
# 형태소 분석기 Okt 불러오기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
from tqdm import tqdm

In [ ]:
nouns = []
for i in tqdm(range(len(data))):
  nouns.append(okt.nouns(data['spell_check'][i]))  

In [ ]:
print(len(nouns))

In [ ]:
print(nouns[:2])

In [ ]:
# custom한 방식으로 불용어 설정
stopwords = "이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 저 전 난 일 걸 뭐 줄 만 건 분 개 끝 잼 이거 번 중 듯 때 게 내 말 나 수 거 점 것 의 가 이 은 들 는 좀 잘 걍 과 도 를 으로 자 에 와 한 하다 을 아 그"
stopwords = list(set(stopwords.split(" ")))
print(stopwords)

In [ ]:
okt.nouns("나는 대학생이다") # 리스트 형태로 저장 

In [ ]:
# 다시 새롭게 추출 
nouns_new = []
for review in nouns:
  temp = []
  for element in review:
    if (element not in stopwords): # and len(element) > 1: # 불용어와 제거
      temp.append(element)
  nouns_new.append(temp)

In [ ]:
print(nouns_new[:2])

## **gensim을 이용한 토픽 모델링 전처리**

In [ ]:
texts = nouns_new.copy()

In [ ]:
print(texts[0])

In [ ]:
from collections import Counter
nouns_counter = Counter(texts[0])
top_nouns = dict(nouns_counter.most_common(50))
top_nouns

In [ ]:
from gensim import corpora
kr_dictionary = corpora.Dictionary(nouns_new) # 단어들의 사전 만들기 (정수 인코딩)
print(kr_dictionary)
# 출현빈도가 적거나 자주 등장하는 단어는 제거 
kr_dictionary.filter_extremes(no_below=5, no_above=0.20)

corpus = [kr_dictionary.doc2bow(text) for text in texts] # Term Document Frequency 만들기, 단어가 해당 문서에서 몇 번 출현하는지 여부
print(corpus[0]) # 수행된 결과에서 첫번째 뉴스 출력. 첫번째 문서의 인덱스는 0

의미: 인코딩이 1번으로된 단어는 해당 문서에서 4번 출현

In [ ]:
print(kr_dictionary)

In [ ]:
print(kr_dictionary[0], kr_dictionary[1], kr_dictionary[2])

In [ ]:
print(corpus[1])

In [ ]:
print(kr_dictionary[5], kr_dictionary[11], kr_dictionary[18])

In [ ]:
data['spell_check'].iloc[0]

In [ ]:
data['spell_check'].iloc[1]

In [ ]:
# 손쉽게 읽을 수 있는 형태로 변환
[[(kr_dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]]

## **토픽 모델 구축하기**

In [ ]:
import gensim

* num_topics : 가설로 정한 토픽의 갯수
* chunksize : 얼마나 많은 문서가 훈련 알고리즘에 사용되는지
(만약에 빠른 학습이 중요하다면, 청크사이즈를 증가)
그러나 Chunksize는 모델 품질에 영향을 미치칠 수 있음
* passes : 패스는 모델 학습시 전체 코퍼스에서 모델을 학습시키는 빈도를 제어
* iteration : 각각 문서에 대해서 루프를 얼마나 돌리는지를 제어
* alpha, eta = auto, 디리클레 분포에 대한 파라미터

https://radimrehurek.com/gensim/models/ldamodel.html#gensim.models.ldamodel.LdaModel

In [ ]:
# LDA 모델 생성 - 대략 
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                           num_topics = 10, 
                                           id2word=kr_dictionary,
                                           random_state=2022,
                                           passes=5,
                                           iterations=50
                                           )

In [ ]:
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

In [ ]:
# 나중에 더 자세히 다룰 내용
tm1 = lda_model[corpus[0]] # 첫 번째 문서에는 어떤 토픽이 ?
tm1

In [ ]:
tm2 = lda_model[corpus[1]] # 두 번째 문서에는 어떤 토픽이 ?
tm2

## **토픽 모델 시각화하기 (I)**

In [ ]:
!pip install pyLDAvis==2.1.2 # colab에서는 최신 버전이 제대로 작동하지 않는 경우가 있음

In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim

In [ ]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, kr_dictionary)
pyLDAvis.save_html(vis, 'result.html')

In [ ]:
import pyLDAvis.gensim as gensimvis
vis_data = gensimvis.prepare(lda_model, corpus, kr_dictionary, sort_topics=False)
pyLDAvis.display(vis_data)

**원의 크기:** 자주 출현하는 토픽일수록 원의 크기가 증가

**근접한 원들:** 상대적으로 유사한 토픽을 의미 (멀리 떨어져 있을수록 다른 내용을 다루고 있는 토픽들)

**람다(λ)의 의미**

* Values of lambda that are very close to zero will show terms that are more specific for a chosen topic. Meaning that you will see terms that are "important" for that specific topic but not necessarily "important" for the whole corpus. (해당 주제에 특화된 단어들을 표시)

* Values of lambda that are very close to one will show those terms that have the highest ratio between frequency of the terms for that specific topic and the overall frequency of the terms from the corpus. (전체에서 많이 나오는 단어이며 동시에 해당 주제에서도 더 자주 출현하는 단어들)

## **토픽 모델 시각화하기 (II)**

In [ ]:
# for i, topic in enumerate(lda_model.get_topics()):
#   print(i)
#   print(topic)
#   print(len(topic))
#   break

In [ ]:
n_words = 10

topic_words = pd.DataFrame({})

for i, topic in enumerate(lda_model.get_topics()):
    top_feature_ids = topic.argsort()[-n_words:][::-1]
    feature_values = topic[top_feature_ids]
    words = [kr_dictionary[id] for id in top_feature_ids]
    topic_df = pd.DataFrame({'value': feature_values, 'word': words, 'topic': i})
    topic_words = pd.concat([topic_words, topic_df], ignore_index=True)

topic_words.head(10)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
g = sns.FacetGrid(topic_words, col="topic", col_wrap=3, sharey=False)
g.map(plt.barh, "word", "value", color='lightgray',edgecolor ="black")

## **토픽 분포 보기**

In [ ]:
print(corpus[0])

In [ ]:
for i, topic_list in enumerate(lda_model[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

In [ ]:
topic_table = pd.DataFrame()

In [ ]:
# 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
for i, topic_list in enumerate(lda_model[corpus]):
  ## lda_model.per_word_topics가 True면 topic_list[0]을 fasle면 topic_list 전체를 할당
  doc = topic_list[0] if lda_model.per_word_topics else topic_list
  #print(doc)           
  doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
  #print(doc)
  for j, (topic_num, prop_topic) in enumerate(doc): # 몇 번 토픽인지와 비중을 나눠서 저장
    if j == 0:
      ## 정렬을 한 상태이므로 가장 앞이 비중이 높음
      topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_list]), ignore_index = True)
    else:
      break

In [ ]:
topic_table

In [ ]:
topic_table = topic_table.reset_index()
topic_table.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topic_table[:10]

**성능을 개선하기 위해선?**

* 명사 뿐만 아니라 형용사, 동사도 사용
* 모델에 집어넣기 전에 너무 자주 나오는 단어 필터링
* 최적의 토픽 개수 (Coherence Model)

In [ ]:
topic_modeling_results = lda_model[corpus]
results = list(topic_modeling_results)
print(results)

In [ ]:
corpus_topics = [sorted(topics, key=lambda record: -record[1])[0] for topics in results]
print(corpus_topics) # 가장 dominant한 topic 만 추출해서 리스트로 저장
print(len(corpus_topics))

In [ ]:
topics = [[(term, round(wt, 3)) for term, wt in lda_model.show_topic(n, topn=20)] for n in range(0, lda_model.num_topics)]
print(topics)

In [ ]:
# topci-term matrix
topics_df = pd.DataFrame([[term for term, wt in topic] for topic in topics], columns = ['Term'+str(i) for i in range(1, 21)],
                         index=['Topic '+str(t) for t in range(1, lda_model.num_topics+1)]).T
topics_df.head(20)

In [ ]:
topics_df2 = pd.DataFrame([', '.join([term for term, wt in topic]) for topic in topics], columns = ['Terms per Topic'],
                         index=['Topic'+str(t) for t in range(1, lda_model.num_topics+1)] )
topics_df2

In [ ]:
# create a dataframe 
corpus_topic_df = pd.DataFrame()

corpus_topic_df['Dominant Topic'] = [item[0] for item in corpus_topics]
corpus_topic_df['Contribution %'] = [round(item[1]*100, 2) for item in corpus_topics]
corpus_topic_df['Topic Terms'] = [topics_df2.iloc[t[0]]['Terms per Topic'] for t in corpus_topics]

corpus_topic_df.head()

In [ ]:
len(corpus_topic_df)

In [ ]:
dominant_topic_df = corpus_topic_df.groupby('Dominant Topic').agg(
                                  Doc_Count = ('Dominant Topic', np.size),
                                  Total_Docs_Perc = ('Dominant Topic', np.size)).reset_index()

dominant_topic_df['Total_Docs_Perc'] = dominant_topic_df['Total_Docs_Perc'].apply(lambda row: round((row*100) / len(corpus), 2))

dominant_topic_df

## **토픽의 갯수 정하기 (== 토픽 모델 평가)**

Coherence는 주제의 일관성을 측정.\
토픽이 얼마나 의미적으로 일관성이 높으지 평가\
토픽 모델이 모델링이 잘 되었을수록 한 주제 안에는 의미론적으로 유사한 단어가 모이게 됨.\
상위 단어 간의 유사도를 계산하면 실제로 해당 주제가 의미론적으로 일치하는 단어들끼리 모여있는지 알 수 있음

- 토픽이 얼마나 의미론적으로 일관성 있는지.
- 높을수록 의미론적 일관성 높음

Perplexity(혼란도)는 확률 모델이 실제 관측되는 값을 얼마나 잘 예측하는지를 평가\
값이 작을수록 토픽 모델이 문서를 잘 반영 (최적의 토픽을 결정할 때 사용 -> 최소점) 

In [ ]:
from gensim.models import CoherenceModel
coherence_model = CoherenceModel(model = lda_model, texts = texts, dictionary = kr_dictionary, coherence = 'c_v')

In [ ]:
coherence_model.get_coherence()

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
coherence_values = []
perplexities=[] # 낮을 수록 더 좋은 값
model_list = []
for i in range(2,21):
  ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = i, id2word=kr_dictionary, passes=5,random_state=2022, iterations=50)
  model_list.append(ldamodel)
  coherencemodel = CoherenceModel(model=ldamodel, texts=texts, dictionary=kr_dictionary, coherence='c_v')
  coherence_values.append(coherencemodel.get_coherence())
  perplexities.append(ldamodel.log_perplexity(corpus))

In [ ]:
coherence_values

최적의 토픽은 18개

In [ ]:
x = range(2, 21, 1)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("Coherence"), loc='best')
plt.show()

In [ ]:
x = range(2, 21, 1)
plt.plot(x, perplexities)
plt.xlabel("Num Topics")
plt.ylabel("Perplexity")
plt.show()

In [ ]:
model_list

In [ ]:
topics = model_list[-3].print_topics(num_words=10)
for topic in topics:
    print(topic)

In [ ]:
vis = pyLDAvis.gensim.prepare(model_list[-3], corpus, kr_dictionary)
pyLDAvis.save_html(vis, 'result_better.html')

## **새로운 문서의 토픽 할당**

In [ ]:
# 훈련에 사용된 corpus가 아닌 새로운 document에 대해 토픽 모델링을 수행할 수 있으나
# 정확도는 다소 떨어질 수 있다.
def doc_to_bow(doc): #
    token = okt.nouns(doc) # 명사만 추출
    temporary = []
    for t in token:
      if t not in stopwords:
        temporary.append(t)
    bow = kr_dictionary.doc2bow(temporary)
    return bow

In [ ]:
result = model_list[-3][(doc_to_bow('업데이트가 느려서 불편해요'))]
print(result)

In [ ]:
result = model_list[-3][(doc_to_bow('미국 영국 독일'))] # 전혀 상관없는 문장의 경우,균등하게 나눈 토픽으로 판단
print(result)